In [10]:
from ipyleaflet import Map, Marker, Icon, CircleMarker, MarkerCluster, AwesomeIcon, Popup, basemaps, basemap_to_tiles
from ipywidgets import HTML

import pandas as pd
import map_methods
import math

In [2]:
city_df = pd.read_csv("../podatoci/mkcities.csv")
city_df['city'] = city_df.city_cyrilic
city_df

,id,city_cyrilic,admin_name_cyrilic,city_latin,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population
0,1807600615,Скопје,Скопје,Skopje,Скопје,Skopje,42.0000,21.4335,Macedonia,MK,MKD,Skopje,False,494087.0
1,1807020286,Тетово,Тетово,Tetovo,Тетово,Tetovo,42.0092,20.9701,Macedonia,MK,MKD,Tetovo,False,119132.0
2,1807488567,Битола,Битола,Bitola,Битола,Bitola,41.0391,21.3395,Macedonia,MK,MKD,Bitola,False,86528.0
3,1807691189,Сопиште,Сопиште,Sopište,Сопиште,Sopiste,41.9547,21.4275,Macedonia,MK,MKD,Sopište,False,NaN
4,1807895120,Кичево,Кичево,Kičevo,Кичево,Kicevo,41.5127,20.9589,Macedonia,MK,MKD,Kičevo,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,1807953626,Белишта,Дебарца,Belčišta,Белишта,Belcista,41.3028,20.8303,Macedonia,MK,MKD,Debarca,False,NaN
67,1807782793,Старо Нагоричане,Старо Нагоричане,Staro Nagoričane,Старо Нагоричане,Staro Nagoricane,42.1981,21.8286,Macedonia,MK,MKD,Staro Nagoričane,False,NaN
68,1807831162,Македонски Брод,Македонски Брод,Makedonski Brod,Македонски Брод,Makedonski Brod,41.5136,21.2153,Macedonia,MK,MKD,Makedonski Brod,False,NaN
69,1807936735,Новаци,Новаци,Novaci,Новаци,Novaci,41.0420,21.4587,Macedonia,MK,MKD,Novaci,False,NaN


In [3]:
infected_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=0&single=true&output=csv", 
                          names=['city', 'infected_in', 'date', 'count', 'age'], header=0)

infected_df

,city,infected_in,date,count,age
0,Скопје,Италијa,2020-02-26,1,50-59
1,Дебар,Италија,2020-03-06,2,60-69
2,Дебар,Македонија,2020-03-09,2,60-69
3,Скопје,Италија,2020-03-09,1,50-59
4,Дебар,Македонија,2020-03-09,1,40-49
5,Дебар,Македонија,2020-03-11,2,60-69
6,Дебар,Македонија,2020-03-13,4,50-59
7,Скопје,Шпанија,3/14/2020,1,непознато
8,Дебар,Македонија,3/14/2020,5,непознато


In [4]:
healed_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=757629356&single=true&output=csv",
                       names=['city', 'date', 'count', 'age'], header=0)
healed_df

,city,date,count,age


In [5]:
dead_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=1754038661&single=true&output=csv",
                                               header=0, names=['city', 'date', 'count', 'age'])
dead_df

,city,date,count,age


In [6]:
infected_by_city = infected_df[['city', 'count']].copy().groupby('city').sum()
healded_by_city = healed_df[['city', 'count']].copy().groupby('city').sum()
dead_by_city = dead_df[['city', 'count']].copy().groupby('city').sum()
merged = pd.merge(infected_by_city, healded_by_city, on='city', how='outer')
merged = pd.merge(merged, dead_by_city, on='city', how='outer')
merged

,count_x,count_y,count
city,,,
Дебар,16,NaN,NaN
Скопје,3,NaN,NaN


In [7]:
merged.fillna(0, inplace=True)
merged = pd.merge(merged, city_df, on='city', how='inner')
merged = merged.set_index("city")
merged

,count_x,count_y,count,id,city_cyrilic,admin_name_cyrilic,city_latin,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population
city,,,,,,,,,,,,,,,,
Дебар,16,0.0,0.0,1807621851,Дебар,Дебар,Debar,Debar,41.5244,20.5242,Macedonia,MK,MKD,Debar,False,NaN
Скопје,3,0.0,0.0,1807600615,Скопје,Скопје,Skopje,Skopje,42.0000,21.4335,Macedonia,MK,MKD,Skopje,False,494087.0


In [19]:


m = map_methods.get_macedonia_map()

def hide_marker(name, row):
    infected = row['count_x']
    healed = row['count_y']
    dead = row['count']
    return (infected - healed - dead) < 1
    

def get_marker(name, location, row):
    infected = row['count_x']
    healed = row['count_y']
    dead = row['count']
    current = infected - healed - dead
    scale = math.log(current, 2)
    return CircleMarker(location=location, radius=(int)(5*scale), color='red', weight=3)

map_methods.add_marker_layer(m=m, d=merged, 
                             cols=["count_x", "count_y", "count", "population"], 
                             names=["Заразени", "Излечени" ,"Мртви", "Население"],
                            get_marker=get_marker, hide_marker=hide_marker)

m

Map(center=[41.6086, 21.7453], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

NameError: name 'hospital_data' is not defined